In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    RocCurveDisplay,
    DetCurveDisplay,
    PrecisionRecallDisplay,
)

from shipsnet.data import load_data_from_json
from shipsnet.viz import array_to_rgb_image

In [ ]:
data_dict = load_data_from_json()

dataframe = pd.DataFrame(data_dict)
dataframe.info()
dataframe.head()

In [ ]:
pixels = np.array(data_dict["data"], dtype=float).reshape(-1, 3, 80, 80)
lon, lat = np.array(data_dict["locations"]).T
labels = np.array(data_dict["labels"], dtype=bool)#.astype(bool)

In [ ]:
fig, axes = plt.subplots(3, 4)

for array, ax in zip(np.random.permutation(pixels), axes.flatten()):
    ax.imshow(array_to_rgb_image(array))
    ax.set_axis_off()
    
fig.tight_layout()
plt.show()

In [ ]:
inputs = pixels / 255 - 0.5
X, y = inputs.reshape(-1, 3 * 80 * 80), labels.T

In [ ]:
# Check reproducibility
def reproducible_train(seed) -> list[np.ndarray]:
    rng = np.random.RandomState(seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=rng, shuffle=True)
    
    model = MLPClassifier(
        hidden_layer_sizes=(10,),
        shuffle=True,
        random_state=rng,
        early_stopping=True,
    )
    trained_model = model.fit(X_train, y_train)

    return trained_model.coefs_ + trained_model.intercepts_

seed = 123456789

params_1 = reproducible_train(seed)
params_2 = reproducible_train(seed)

assert all([np.allclose(a, b) for a, b in zip(params_1, params_2)])

In [ ]:
seed = 1234567
rng = np.random.RandomState(seed)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=rng, shuffle=True)

model = MLPClassifier(
    hidden_layer_sizes=(10,),
    activation="relu",
    solver="adam",
    batch_size=8,
    max_iter=50,
    shuffle=True,
    random_state=rng,
    verbose=True,
    early_stopping=True,
)
trained_model = model.fit(X_train, y_train)

In [ ]:
y_predicted_classes = trained_model.predict(X_test)
y_predicted_probs = trained_model.predict_proba(X_test)[:, 1]

fig, axes = plt.subplots(2, 2, figsize=(8, 8))

for metric, y_pred, ax in zip(
    [ConfusionMatrixDisplay, RocCurveDisplay, DetCurveDisplay, PrecisionRecallDisplay],
    [y_predicted_classes, y_predicted_probs, y_predicted_probs, y_predicted_probs],
    axes.flatten()
):
    disp = metric.from_predictions(y_test, y_pred, ax=ax)
    ax.set_title(metric.__name__.replace("Display", ""))

fig.tight_layout()